# Neural Circuit Policies

Neural Circuit Policies are recurrent neural network models inspired by the nervious system of the nematode C. elegans. Compared to standard ML models, NCPs have:

1. Neurons that are modelled by an Ordinary Differential Equation
2. A sparse structured wiring

_The goal of this notebook is to understand ncps and learning how to implement for our task. For this we will refer to the github package ncps by Mathias Lechner._

In [1]:
# Pytorch Example
import torch
from ncps.torch import CfC

# a fully connected CfC network
rnn = CfC(input_size=20, units=50)
x = torch.randn(2, 3, 20) # (batch, time, features)
h0 = torch.zeros(2,50) # (batch, units)
output, hn = rnn(x,h0)

In [3]:
# Tensorflow example
import tensorflow as tf
from ncps.tf import LTC
from ncps.wirings import AutoNCP

wiring = AutoNCP(28, 4) # 28 neurons, 4 outputs
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(None, 2)),
        # LTC model with NCP sparse wiring
        LTC(wiring, return_sequences=True),
    ]
)

## Neuron Models

The package currently provides two neuron models: LTC and CfC:

1. [Liquid time-constant (LTC)]() model \
   It is based on neurons based on differential equations interconnected via sigmoidal synapses. The term liquid time comes from the property of LTCs that their timing behaviour is adaptive to the input. How fast or slow they respond to some stimulus can depend on the specific input. Because LTCs are Ordinary Differential Equations, their behaviour can only be described over time. LTCs are universal approximators and implement causal dynamical models. However, the LTC model has one major disadvantage: to compute their output, we need a numerical differential-equation solver which seriously slows down their training and inference time. CfC model can resolve this bottleneck.

2. [Closed-form continuous-time (CfC)]() model \
   CfC models resolve the bottleneck by approximating the closed form solution of the differential equation.


_Both the LTC and the CfC models are recurrent neural networks and possess a temporal state. Therefore, these models are only applicable to sequential or time series data._

## Wirings

- Fully Connected
- Sparse structured wiring
  1. Random
  2. Neural Circuit Policy

![Wirings](wirings.webp)

#### Fully Connected Wiring
We can use both the models described above with a fully-connected wiring diagram by simply passing the number of neurons, as it is done in standard ML models such as LSTMs, GRUs, MLPs and Transformers.

In [4]:
from ncps.torch import CfC

# a fully connected CfC network
rnn = CfC(input_size=20, units=50)

#### Sparse Structured Wiring

We can also specify sparse structured wirings in the form of a ```ncps.wirings.Wiring``` object. 

#### NCP
The most interesting wiring paradigm provided in the package is the Neural Circuit Policy. It comprises of a 4-layer recurrent connection principle of the following neurons:
1. Sensory
2. Inter
3. Command
4. Motor

The easiest way to create an NCP wiring is using the ```AutoNCP``` class, which requires defining the total number of neurons and the number of motor neurons (= output size).

In [5]:
from ncps.torch import CfC
from ncps.wirings import AutoNCP

wiring = AutoNCP(28,4)   # 28 neurons, 4 outputs
input_size = 20
rnn = CfC(input_size, wiring)

![Code Diagram](code_diagram.webp)

## References
1. [ncps - Mathias Lechner Github Repo](https://github.com/mlech26l/ncps)
2. [NCPs Docs](https://ncps.readthedocs.io/en/latest/index.html)